In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

In [2]:
spark = SparkSession.builder \
    .appName("Google Play Store Spark") \
    .getOrCreate()

In [3]:
df = spark.read.csv('googleplaystore.csv', header=True, inferSchema=True, sep=",")

In [4]:
df.count()

10841

In [5]:
df.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 

In [6]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [7]:
# Data cleaning
df = df.drop("size", "Content Rating", "Last Updated", "Android Ver", "Current Ver")

# Reviews table
df = df.withColumn("Reviews", df["Reviews"].cast(IntegerType()))

# Rating table
df = df.withColumn("Rating", func.when(df["Rating"] == "NaN", 0).otherwise(df["Rating"])) \
    .withColumn("Rating", func.col("Rating").cast(DoubleType()))

# Installs table
df = df.withColumn("Installs", func.regexp_replace("Installs", "\+" , "")) \
    .withColumn("Installs", func.regexp_replace("Installs", ",", "")) \
    .withColumn("Installs", func.col("Installs").cast(IntegerType()))

# Price table
df = df.withColumn("Price", func.regexp_replace("Price", "^\\$", "")) \
    .withColumn("Price", func.col("Price").cast(DoubleType()))

df.show()


+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|  0.0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|  0.0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|  0.0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|  0.0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|  0.0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|  0.0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|  0.0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

In [8]:
# 1. Find out top 10 reviews given to the apps
df.groupBy("App").agg(func.sum("Reviews").alias("total_reviews")) \
  .orderBy(func.desc("total_reviews")) \
  .show(10)

+--------------------+-------------+
|                 App|total_reviews|
+--------------------+-------------+
|           Instagram|    266241989|
|  WhatsApp Messenger|    207348304|
|      Clash of Clans|    179558781|
|Messenger – Text ...|    169932272|
|      Subway Surfers|    166331958|
|    Candy Crush Saga|    156993136|
|            Facebook|    156286514|
|         8 Ball Pool|     99386198|
|        Clash Royale|     92530298|
|            Snapchat|     68045010|
+--------------------+-------------+
only showing top 10 rows



In [9]:
# 2. Top 10 installs apps and distribution of type (free/paid)
df.groupBy("App").agg(func.sum("Installs").alias("total_installs")) \
    .orderBy(func.desc("total_installs")) \
    .show(10)

+--------------------+--------------+
|                 App|total_installs|
+--------------------+--------------+
|      Subway Surfers|    6000000000|
|           Instagram|    4000000000|
|        Google Drive|    4000000000|
|            Hangouts|    4000000000|
|         Google News|    4000000000|
|       Google Photos|    4000000000|
|    Candy Crush Saga|    3500000000|
|Messenger – Text ...|    3000000000|
|Google Chrome: Fa...|    3000000000|
|Maps - Navigate &...|    3000000000|
+--------------------+--------------+
only showing top 10 rows



In [10]:
# 3. Category wise distribution of installed apps
df.groupBy("Category").agg(func.sum("Installs").alias("total_installs")) \
    .orderBy(func.desc("total_installs")) \
    .show()

+-------------------+--------------+
|           Category|total_installs|
+-------------------+--------------+
|               GAME|   35086024415|
|      COMMUNICATION|   32647276251|
|       PRODUCTIVITY|   14176091369|
|             SOCIAL|   14069867902|
|              TOOLS|   11452771915|
|             FAMILY|   10258263505|
|        PHOTOGRAPHY|   10088247655|
| NEWS_AND_MAGAZINES|    7496317760|
|   TRAVEL_AND_LOCAL|    6868887146|
|      VIDEO_PLAYERS|    6222002720|
|           SHOPPING|    3247848785|
|      ENTERTAINMENT|    2869160000|
|    PERSONALIZATION|    2325494782|
|BOOKS_AND_REFERENCE|    1921469576|
|             SPORTS|    1751174498|
| HEALTH_AND_FITNESS|    1582072512|
|           BUSINESS|    1001914865|
|            FINANCE|     876648734|
|          EDUCATION|     871452000|
|MAPS_AND_NAVIGATION|     719281890|
+-------------------+--------------+
only showing top 20 rows



In [11]:
# 4. Top paid apps
df.filter(df["Type"] == "Paid").groupBy("App") \
    .agg(func.sum("Price").alias("total_price")) \
    .orderBy(func.desc(func.col("total_price"))) \
    .show()

+--------------------+-----------+
|                 App|total_price|
+--------------------+-----------+
|I'm Rich - Trump ...|      400.0|
|  I AM RICH PRO PLUS|     399.99|
|   I Am Rich Premium|     399.99|
|      I am Rich Plus|     399.99|
|I'm Rich/Eu sou R...|     399.99|
|most expensive ap...|     399.99|
|       I Am Rich Pro|     399.99|
|  I am rich(premium)|     399.99|
|           I am Rich|     399.99|
|          I am Rich!|     399.99|
|         💎 I'm rich|     399.99|
|I am rich (Most e...|     399.99|
|           I am rich|     399.99|
|         Eu Sou Rico|     394.99|
|           I Am Rich|     389.99|
| I am extremely Rich|     379.99|
|       I am rich VIP|     299.99|
|        EP Cook Book|      200.0|
|Vargo Anesthesia ...|     159.98|
|       cronometra-br|     154.99|
+--------------------+-----------+
only showing top 20 rows



In [12]:
# 5. Top paid rating apps
rating_top = df.filter(df["Type"] == "Paid").orderBy(df["Type"].desc()).select("Rating").first()[0]
df.filter((df["Type"] == "Paid") & (df["Rating"] == rating_top)) \
    .select("App", "Rating") \
    .show(truncate=False)

+-------------------------------------------------+------+
|App                                              |Rating|
+-------------------------------------------------+------+
|TurboScan: scan documents and receipts in PDF    |4.7   |
|TurboScan: scan documents and receipts in PDF    |4.7   |
|Fuzzy Numbers: Pre-K Number Foundation           |4.7   |
|Toca Life: City                                  |4.7   |
|Toca Life: Hospital                              |4.7   |
|Toca Life: City                                  |4.7   |
|Children Educational Game Full                   |4.7   |
|Lanterns: The Harvest Festival                   |4.7   |
|Tsuro - The Game of the Path                     |4.7   |
|OmniMedix Medical Calculator                     |4.7   |
|Navi Radiography Pro                             |4.7   |
|C4droid - C/C++ compiler & IDE                   |4.7   |
|Fuzzy Numbers: Pre-K Number Foundation           |4.7   |
|Q Alerts: QAnon Drop Notifications, Research +++ |4.7  